In [ ]:
import sys
sys.path.append("..")
from dataset.dataloader import MyOcrDataloader
import pandas as pd
import math
import random
import wandb
import torch
import gc
import yaml
import os
from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
from utils.utils import forward_pass_with_labels, update
from jiwer import wer, cer
from transformers import AutoFeatureExtractor, DeiTForImageClassificationWithTeacher
from tqdm.auto import tqdm
from utils.utils import *

In [2]:
# config_path = os.path.join(os.path.abspath(os.path.join(os.getcwd(), '..')), "config/main.yaml")
# with open(config_path, "r") as file:
#     config = yaml.safe_load(file)

# task  = config["TRAIN_TASK"]
# TRAIN_PATH =config[task+"_"+"TRAIN_PATH"]
# VAL_PATH =config[task+"_"+"VAL_PATH"]
# IMG_ROOT = config[task+"_"+"IMG_ROOT"]

# MODEL_ID = config["MODEL_ID"]
# MODEL_ID = "facebook/deit-base-distilled-patch16-384"

# device = "cuda" if torch.cuda.is_available() else "cpu"
# # processor = TrOCRProcessor.from_pretrained(MODEL_ID)
# processor = AutoFeatureExtractor.from_pretrained('facebook/deit-base-distilled-patch16-384')
# # model =VisionEncoderDecoderModel.from_pretrained(MODEL_ID).to(device)
# model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
#     "facebook/deit-base-distilled-patch16-384", "rasyosef/bert-medium-amharic", from_tf =True, cache_dir = "/home/ubuntu/data/cache_dir"
# ).to(device)
# model.config.decoder_start_token_id = processor.tokenizer.eos_token_id
# model.config.pad_token_id = processor.tokenizer.pad_token_id
# model.config.vocab_size = model.config.decoder.vocab_size

In [ ]:
config_path = os.path.join(os.path.abspath(os.path.join(os.getcwd(), '..')), "config/main.yaml")
with open(config_path, "r") as file:
    config = yaml.safe_load(file)

task  = config["TRAIN_TASK"]
TRAIN_PATH =config[task+"_"+"TRAIN_PATH"]
VAL_PATH =config[task+"_"+"VAL_PATH"]
IMG_ROOT = config[task+"_"+"IMG_ROOT"]

MODEL_ID = config["MODEL_ID"]
MODEL_ID = "facebook/deit-base-distilled-patch16-384"

device = "cuda" if torch.cuda.is_available() else "cpu"
# processor = TrOCRProcessor.from_pretrained(MODEL_ID)
processor = AutoFeatureExtractor.from_pretrained('facebook/deit-base-distilled-patch16-384')
# model =VisionEncoderDecoderModel.from_pretrained(MODEL_ID).to(device)
tokenizer = AutoTokenizer.from_pretrained("rasyosef/bert-medium-amharic")
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    "facebook/deit-base-distilled-patch16-384", "rasyosef/bert-medium-amharic", from_tf =True, cache_dir = "/home/ubuntu/data/cache_dir"
).to(device)
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.decoder.config.vocab_size
model.generation_config.decoder_start_token_id = tokenizer.cls_token_id

In [4]:
## sanity check for data loader
os.chdir("../dataset")
train_data = MyOcrDataloader(TRAIN_PATH, preprocessor=processor, tokenizer=tokenizer, img_root=IMG_ROOT)
val_data = MyOcrDataloader(VAL_PATH, preprocessor=processor, tokenizer  = tokenizer, img_root=IMG_ROOT)
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    batch_size  = config['BATCH_SIZE'],
    shuffle     = True,
    collate_fn= val_data.collate_fn
    )
val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    batch_size  = config['BATCH_SIZE'],

    shuffle     = False,
    collate_fn= train_data.collate_fn
    )



In [ ]:
for batch in train_loader:
    print(batch[0].shape, batch[1].shape,tokenizer.batch_decode(batch[1], skip_special_tokens=True))
    
    break

In [6]:
# for batch in train_loader:
#     # print(batch[0].shape, batch[1].shape,)
#     labels = tokenizer.batch_decode(batch[1], skip_special_tokens=True)

   

#     # Generate text
#     generated_ids = model.generate(batch[0].cuda())

#     # Decode the generated text
#     generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

#     print(generated_text, labels[0])  
#     break

In [7]:
# import matplotlib.pyplot as plt

# # Fetch a single batch
# train_wers = 0
# train_cers = 0
# samples = 0

# for batch in train_loader:
#     # Get the images and labels
#     images = batch[0]
#     labels = tokenizer.batch_decode(batch[1], skip_special_tokens=True)

#     # Move images to GPU (if available)
#     images = images.cuda()

#     # Generate text using the model
#     generated_ids = model.generate(images)
#     generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

#     for pred, actual in zip(generated_texts, labels):
#         train_wers += wer(actual, pred)
#         train_cers += cer(actual, pred)

#     samples += len(generated_texts)
#     # Plot the images and corresponding texts
#     fig, axes = plt.subplots(1, len(images), figsize=(15, 5))
#     print(train_wers, train_cers)
#     if len(images) == 1:
#         axes = [axes]  # Ensure axes is iterable even for a single image

#     for idx, (image, gt_text, pred_text) in enumerate(zip(images, labels, generated_texts)):
#         # Convert tensor to numpy and permute dimensions for plotting
#         image = image.cpu().permute(1, 2, 0).numpy()

#         # Display the image
#         axes[idx].imshow(image)
#         axes[idx].set_title(f"GT: {gt_text}\nPred: {pred_text}")
#         axes[idx].axis('off')  # Hide axes
#     break
#     plt.tight_layout()
#     plt.show()
#     # break

In [8]:
def one_train_epoch(model, train_data_loader, scaler, optimizer):
    model.train()
    pbar = tqdm(range(len(train_data_loader)))
    train_loss= 0.0
    
    for i,batch in enumerate(train_data_loader):
        loss, logits = update(model, scaler, optimizer, batch)
        preds = tokenizer.batch_decode(logits.argmax(dim=2), skip_special_tokens=True)
        actuals = tokenizer.batch_decode(batch[1], skip_special_tokens=True)

        train_loss += (1/(i+1))*(loss-train_loss)
        perplexity          = torch.exp(torch.tensor(train_loss))
        # print(logits.shape,"preds", preds, "\n", "="*20, "\nactuals", actuals)
        # for pred, actual in zip(preds, actuals):
        #     train_wers += wer(actual, pred)
        #     train_cers += cer(actual, pred)

   
        pbar.set_description(f"Train Loss: {train_loss:.4f} Perplexity: {perplexity:.4f}")
        pbar.update(1)

    pbar.close()
    return train_loss, perplexity

In [9]:
def validation (model, val_data_loader):
    model.eval()
    pbar = tqdm(range(len(val_data_loader)))
    running_distance = 0.0
    running_cer = 0.0
    running_wer = 0.0
    running_char_f1 = 0.0
    running_word_f1 =0.0
    samples = 0.0
    for i,batch in enumerate(val_data_loader):
        images = batch[0].to(model.device)

        with torch.inference_mode():
            generated_ids = model.generate(images)
            
        generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        actuals = tokenizer.batch_decode(batch[1], skip_special_tokens=True)



        # print(logits.shape,"preds", preds, "\n", "="*20, "\nactuals", actuals)
        # for pred, actual in zip(preds, actuals):
        #     train_wers += wer(actual, pred)
        #     train_cers += cer(actual, pred)
        dist, cer, wer, charf1, word_f1 = compute_metrics(generated_texts, actuals)
        running_distance += dist
        running_cer += cer
        running_wer += wer
        running_char_f1 += charf1
        running_word_f1 += word_f1
        samples += len(generated_ids)
   
        pbar.set_description(f"Val WER: {running_wer/samples:.4f} CER: {running_cer/samples:.4f} LevDistance: {running_distance/samples:.4f}  Charf1: {running_char_f1/samples:.4f} word_f1: {running_word_f1/samples:.4f}")
        pbar.update(1)
    
    print("pred", generated_texts[:2])
    print("actual", actuals[:2])
    pbar.close()
    return running_distance/samples, running_cer/samples, running_wer/samples, running_char_f1/samples, running_word_f1/samples

In [10]:
# def train(model, tdataloader,vdataloader, optimizer, scaler, epochs=3, log_dir="logs/a"):
#     os.makedirs(log_dir, exist_ok=True)
#     with open(f"{log_dir}/train.txt", "w") as tf, open(f"{log_dir}/val.txt", "w") as vf:
#         tf.write(f"Epoch,Loss,Score\n")
#         vf.write("Epoch,Loss,Score\n")
#     best_val_score = 0    
#     train_loss = 0
#     for epoch in range(1, 1+epochs):
#         loss, perplexity  = one_train_epoch(model, tdataloader, optimizer, scaler )
#         levenshtein_distance, cer, wer, charf1, wordf1 = validation(model, val_loader)
#         # with open(f"{log_dir}/train.txt", "a+") as tf, open(f"{log_dir}/val.txt", "a+") as vf:
#         #     tf.write(f"{epoch},{train_loss},{tscore}\n")
#         #     vf.write(f"{epoch},{val_loss},{score}\n")

In [11]:

# optimizer = torch.optim.AdamW(model.parameters(), 1e-4)
# optimizer.zero_grad()




# scaler =  torch.cuda.amp.GradScaler(enabled=True)

# train(model, train_loader, val_loader,optimizer, scaler, epochs=15,log_dir="logs/vit_amharic_bert")

In [ ]:
''' defining optimizer '''
# vocab_size = len(tokenizer)
# weights = torch.ones(vocab_size).to("cuda")  # default weight = 1 for all
# whitespace_token_id = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(' ')[0])
# # Decrease weight for whitespace token
# weights[whitespace_token_id] = 0.1  # e.g., reduce impact by 90%
loss_func   = torch.nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
scaler      = torch.cuda.amp.GradScaler()
if config["optimizer"] == "SGD":
  # feel free to change any of the initializations you like to fit your needs
  optimizer = torch.optim.SGD(model.parameters(),
                              lr=config["learning_rate"],
                              momentum=config["momentum"],
                              weight_decay=1E-4,
                              nesterov=config["nesterov"])

elif config["optimizer"] == "Adam":
  # feel free to change any of the initializations you like to fit your needs
  optimizer = torch.optim.Adam(model.parameters(),
                               lr=float(config["learning_rate"]),
                               weight_decay=1e-4)

elif config["optimizer"] == "AdamW":
  # feel free to change any of the initializations you like to fit your needs
  optimizer = torch.optim.AdamW(model.parameters(),
                                lr=float(config["learning_rate"]),
                                weight_decay=0.01)

''' defining scheduler '''

if config["scheduler"] == "ReduceLR":
  #Feel Free to change any of the initializations you like to fit your needs
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                factor=config["factor"], patience=config["patience"], min_lr=1E-8, verbose=True)

elif config["scheduler"] == "CosineAnnealing":
  #Feel Free to change any of the initializations you like to fit your needs
  scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                T_max = 35, eta_min=1E-8)

In [ ]:
# using WandB? resume training?

USE_WANDB = config["USE_WANDB"]
RESUME_LOGGING = False

# creating your WandB run
run_name = "{}_Transformer_ENC-{}/{}_DEC-{}/{}_{}_{}_{}_{}".format(
    config["Name"],
    config["enc_num_layers"],       # only used in Part II with the Transformer Encoder
    config["enc_num_heads"],        # only used in Part II with the Transformer Encoder
    config["dec_num_layers"],
    config["dec_num_heads"],
    config["d_model"],
    config["d_ff"],
    config["optimizer"],
    config["scheduler"])
task = "handonly"

if USE_WANDB:

    wandb.login(key="3c7b273814544590b64c54d9a5242bde38616e02", relogin=True) # TODO enter your key here

    if RESUME_LOGGING:
        run_id = ""
        run = wandb.init(
            id     = run_id,        ### Insert specific run id here if you want to resume a previous run
            resume = True,          ### You need this to resume previous runs, but comment out reinit=True when using this
            project = task+"ocr-cnn-lstm",  ### Project should be created in your wandb account
        )

    else:
        run = wandb.init(
            name    = run_name,     ### Wandb creates random run names if you skip this field, we recommend you give useful names
            reinit  = True,         ### Allows reinitalizing runs when you re-run this cell
            project = task+"ocr-cnn-lstm",  ### Project should be created in your wandb account
            config  = config        ### Wandb Config for your run
        )

        ### Save your model architecture as a string with str(model)
        model_arch  = str(model)

        ### Save it in a txt file
        arch_file   = open("model_arch.txt", "w")
        file_write  = arch_file.write(model_arch)
        arch_file.close()

        ### Log it in your wandb run with wandb.save()
        # wandb.save("model_arch.txt")

In [17]:
e                   = 0
best_loss           = 0
best_distance  = 0
checkpoint_root = os.path.join(os.getcwd(), "checkpoints-basic-cnn-transformer")
os.makedirs(checkpoint_root, exist_ok=True)

USE_WANDB = config["USE_WANDB"]
if USE_WANDB:
    wandb.watch(model, log="all")
task =  config["TRAIN_TASK"]
checkpoint_best_loss_model_filename     = task +'checkpoint-best-loss-model.pth' 
checkpoint_best_distance_model_filename     = task +'checkpoint-best-distance-model.pth'

checkpoint_last_epoch_filename          = 'checkpoint-epoch-'
best_loss_model_path                    = os.path.join(checkpoint_root, checkpoint_best_loss_model_filename)
best_distance_model_path                    = os.path.join(checkpoint_root, checkpoint_best_distance_model_filename)


if RESUME_LOGGING:
    # change if you want to load best test model accordingly
    checkpoint = torch.load(wandb.restore(checkpoint_best_loss_model_filename, run_path=""+run_id).name)

    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    e = checkpoint['epoch']

    print("Resuming from epoch {}".format(e+1))
    print("Epochs left: ", config['epochs']-e)
    print("Optimizer: \n", optimizer)

torch.cuda.empty_cache()
gc.collect()

epochs = config["epochs"]
for epoch in range(e, epochs):

    print("\nEpoch {}/{}".format(epoch+1, config["epochs"]))

    curr_lr = float(optimizer.param_groups[0]["lr"])

    train_loss, train_perplexity = one_train_epoch(model, train_loader, optimizer, scaler )

    print("\nEpoch {}/{}: \nTrain Loss {:.04f}\t Train Perplexity {:.04f}\t Learning Rate {:.04f}".format(
        epoch + 1, config["epochs"], train_loss, train_perplexity, curr_lr))

    if (epoch >1 )and (epoch % 10 == 0):    # validate every 2 epochs to speed up training
        levenshtein_distance, cer, wer, charf1, wordf1 = validation(model, val_loader)
        if best_distance <= cer:
            best_distance = cer
            save_model(model, optimizer, scheduler, ['val_distance', cer], epoch, best_distance_model_path)
            # wandb.save(best_loss_model_path)
            print("Saved distance training model")
        print("Levenshtein Distance {:.04f}".format(levenshtein_distance))
        if USE_WANDB:
            wandb.log({"train_loss"     : train_loss,
                    "train_perplexity"  : train_perplexity,
                    "learning_rate"     : curr_lr,
                    "val_distance"      : levenshtein_distance,
                    "charf1": charf1,
                    "wordf1": wordf1,})

    else:
        if USE_WANDB:

            wandb.log({"train_loss"     : train_loss,
                    "train_perplexity"  : train_perplexity,
                    "learning_rate"     : curr_lr})

    if best_loss >= train_loss:
        best_loss = train_loss
        save_model(model, optimizer, scheduler, ['train_loss', train_loss], epoch, best_loss_model_path)
        # wandb.save(best_loss_model_path)
        print("Saved best training model")


Val WER: 0.9411 CER: 0.8047 LevDistance: 60.9733  Charf1: 0.0593 word_f1: 0.0192:  22%|██▏       | 34/157 [03:52<14:02,  6.85s/it]



Epoch 1/50


Train Loss: 0.2529 Perplexity: 1.2878: 100%|██████████| 157/157 [03:53<00:00,  1.48s/it]



Epoch 1/50: 
Train Loss 0.2529	 Train Perplexity 1.2878	 Learning Rate 0.0001

Epoch 2/50


Train Loss: 0.2391 Perplexity: 1.2701: 100%|██████████| 157/157 [03:51<00:00,  1.48s/it]


...

Train Loss: 0.0707 Perplexity: 1.0732: 100%|██████████| 157/157 [04:01<00:00,  1.54s/it]



Epoch 29/50: 
Train Loss 0.0707	 Train Perplexity 1.0732	 Learning Rate 0.0001

Epoch 30/50


Train Loss: 0.0694 Perplexity: 1.0718: 100%|██████████| 157/157 [03:53<00:00,  1.49s/it]



Epoch 30/50: 
Train Loss 0.0694	 Train Perplexity 1.0718	 Learning Rate 0.0001

Epoch 31/50


Train Loss: 0.0699 Perplexity: 1.0724:  41%|████▏     | 65/157 [01:37<02:14,  1.46s/it]